## Create monthly boundary condition forcing files based on Temperature and Salinity

In [1]:
import netCDF4 as nc
import numpy as np
import xarray as xr

%matplotlib inline

#### Parameters

In [2]:
# domain dimensions:
imin, imax = 1139, 2179
jmin, jmax = 159, 799

# Rimwidths:
rimwidthE  = 10

# Boundary start and end coordinates: (i1,i2,j1,j2)
bdy_NCB = (1598,2177,788,798) # Northern Canada Basin 

Stuff to be careful of: 
- Python indexing starts at 0, Fortran at 1
- Python indexing does not include the end digit, Fortran does

#### Files

In [5]:
# Modified ANHA12 mesh mask file (http://knossos.eas.ualberta.ca/anha/anhatable.php)
mesh         = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20230213.nc')
mesh_lon     = np.array(mesh.variables['nav_lon'])
mesh_lat     = np.array(mesh.variables['nav_lat'])

#### Functions

In [16]:
def load_climatology(year, month):        
    # load climatology of ANHA12 gridT file:
    with xr.open_dataset(f'/data/brogalla/ANHA12/new_averages/gridT_{year}_{month}.nc') as df_gridT:    
        return df_gridT['votemper'].values[:,:,:], df_gridT['vosaline'].values[:,:,:]

In [7]:
def create_AO_BC(bdy_temp, bdy_sal):
    # Pb end-members:
    PML_Pb   = 3e-12   # Polar Mixed Layer Pb
    ACW_Pb   = 10e-12  # Alaskan Coastal Water Pb
    
    PML_temp  = 0      # Polar mixed layer temperature
    ACW_temp  = 2.5    # ACW temperature
    
    # Linear interpolate endpoints between PML and ACW:
    end_temp = [PML_temp, ACW_temp]
    end_dPb = [PML_Pb, ACW_Pb]
    dPb_BC  = np.interp(bdy_temp, end_temp, end_dPb)    
    
    return dPb_BC

In [17]:
def flatten_input(var, order):
    # Flatten dimensions of the array that you supply based on order of either "C" (Python) style or "F" (Fortran) style
    b = var[0,:,:].flatten(order=order)
    for i in range(1,len(var)):
        a = var[i,:,:].flatten(order=order)
        b = np.vstack((b,a))
    return b

In [18]:
def reshape_boundary(rimwidth, boundary_dPb, order):
    # Function that converts the boundary condition array to the dimensions that Fortran is happy with
    
    dPb_O = flatten_input(boundary_dPb, order)
    
    # dimensions: (1 (or time_counter I think), # of depth_levels, 1, boundary_length*rimwidth)
    dPb_OBC = np.reshape(dPb_O, (1,50,1,np.max(boundary_dPb.shape)*rimwidth))

    return dPb_OBC

In [19]:
def save_file(year, month):    
    file_write = xr.Dataset(
        {'dPb_E' : (("time_counter","deptht","y","x1"), dPb_east_BC)}, 
        coords = {
            "time_counter": np.zeros(1),
            "deptht": np.zeros(50),
            "y": np.zeros(1),
            "x1": np.zeros(np.max(dPb_east_BC.shape))
        },
    )
    file_write.to_netcdf(f'/ocean/brogalla/GEOTRACES/data/Pb-tuning-202303/Labrador_Sea/Pb_OBC_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')
    
    return

#### Calculate

In [22]:
for year in range(2002,2003):

    print(year)
    nCB_T = np.zeros((12,50,bdy_NCB[1]-bdy_NCB[0],bdy_NCB[3]-bdy_NCB[2])); 
    nCB_S = np.zeros((12,50,bdy_NCB[1]-bdy_NCB[0],bdy_NCB[3]-bdy_NCB[2]));

    for month in range(1,13):
        temp, sal  = load_climatology(year, f'{month:02}')

        nCB_T[month-1,:,:,:] = temp[:,bdy_NCB[0]:bdy_NCB[1],bdy_NCB[2]:bdy_NCB[3]]
        nCB_S[month-1,:,:,:] = sal[:,bdy_NCB[0]:bdy_NCB[1],bdy_NCB[2]:bdy_NCB[3]]
    
    # Create boundary conditions:
    dPb_nCB  = np.zeros((12,50,bdy_NCB[1]-bdy_NCB[0],bdy_NCB[3]-bdy_NCB[2]))
    for month in range(1,13):
        dPb_nCB[month-1,:,:,:] = create_AO_BC(nCB_T[month-1], nCB_S[month-1])
    
    # fill any NaNs with a reasonable constant value 
    dPb_nCB[np.isnan(dPb_nCB)] = 3e-12
    
    # Flip the order of the layers (necessary for my use case, but depends a bit on what you're doing)
    dPb_nCBr = np.flip(dPb_nCB, axis=3)
    
    for month in range(1,13):
        dPb_east_BC  = reshape_boundary(rimwidthE, dPb_nCBr[month-1], 'F')

#         save_file(year, month)
        
    # Check that there are no NaN values in the files:
    print('East BC # of NaNs: ', sum(dPb_east_BC[np.isnan(dPb_east_BC)]))

2002
East BC # of NaNs:  0


In [23]:
# shape of underlying temperature and salinity variables:
# (depth dimension, i, j)
print(nCB_T[0,:,:,:].shape)

# shape in the boundary condition file:
# Flattened array(1, depth dimension, 1, rimwidth*boundary length) 
print(dPb_east_BC.shape)

(50, 579, 10)
(1, 50, 1, 5790)
